<a href="https://colab.research.google.com/github/pmj-chosim/OSSCA/blob/main/6_Cat_%26_Dogs_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98_%EB%94%A5_%EB%9F%AC%EB%8B%9D_%EB%AA%A8%EB%8D%B8_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1. 사진 구글링으로 다운받기

#2. 사이즈 일괄적으로 맞추기

In [ ]:
from PIL import Image
import os

def resize_crop_images(input_path, output_path, target_size):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for filename in os.listdir(input_path):
        image_path = os.path.join(input_path, filename)
        if os.path.isfile(image_path):
            img = Image.open(image_path)
            img = img.resize(target_size, Image.LANCZOS)
            img.save(os.path.join(output_path, filename))

# 입력 폴더 경로와 출력 폴더 경로 설정
input_path = "C:/Users/parkm/Desktop/저장" # 입력 폴더 경로를 지정하세요.
output_path = "C:/Users/parkm/Desktop/저장" # 출력 폴더 경로를 지정하세요.

# 입력 shape 설정 (150x150으로 변경)
target_size = (150, 150)

resize_crop_images(input_path, output_path, target_size)


#3. 드라이브에 업로드 하기
>좌측 마운트된 드라이브에 바로 업로드

난 ex 폴더 밑에 dog, cat 폴더 파서 각 폴더에 dog,cat 리사이즈된 이미지 저장

ex/

   ├── cat/

   |    ├── cat_image1.jpg

   |    ├── cat_image2.jpg

   |    ├── ...

   |

   └── dog/

        ├── ...

#4. 딥러닝 학습 및 테스트

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# 이미지 크기와 경로 설정
image_size = (150, 150)
train_data_dir = "/content/drive/MyDrive/ex"

# 데이터 전처리와 증강 설정
train_data_gen = ImageDataGenerator(
    rescale=1.0 / 255,  # 픽셀 값을 0과 1 사이로 정규화
    shear_range=0.2,    # 이미지를 잘라내어 각도를 변화시킴
    zoom_range=0.2,     # 이미지를 확대/축소
    horizontal_flip=True  # 이미지를 좌우로 뒤집음
)

# 훈련 데이터 불러오기
train_data = train_data_gen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='binary'  # 이진 분류 (개와 고양이)
)

# 딥러닝 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

# 모델 훈련
model.fit(train_data, epochs=30)

# 테스트 이미지 분류
test_image_path1 = "/content/drive/MyDrive/test1.jpeg"  # 테스트할 이미지 파일 경로
test_image_path2 = "/content/drive/MyDrive/test2.jpeg"  # 테스트할 이미지 파일 경로

def classify_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=image_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # 모델의 입력 형태로 변환
    predictions = model.predict(img_array)
    if predictions[0][0] < 0.5:
        return "고양이"
    else:
        return "개"

result1 = classify_image(test_image_path1)
result2 = classify_image(test_image_path2)
print(f"{test_image_path1}는 {result1}입니다.")
print(f"{test_image_path2}는 {result2}입니다.")


Found 20 images belonging to 3 classes.
Epoch 1/30
1/1 [==============================] - 4s 4s/step - loss: 0.6199 - accuracy: 0.5000
Epoch 2/30
1/1 [==============================] - 1s 1s/step - loss: 0.2283 - accuracy: 0.5000
Epoch 3/30
1/1 [==============================] - 1s 1s/step - loss: -0.1359 - accuracy: 0.5000
Epoch 4/30
1/1 [==============================] - 1s 1s/step - loss: -0.4994 - accuracy: 0.5000
Epoch 5/30
1/1 [==============================] - 1s 1s/step - loss: -0.9010 - accuracy: 0.5000
Epoch 6/30
1/1 [==============================] - 1s 1s/step - loss: -1.3175 - accuracy: 0.5000
Epoch 7/30
1/1 [==============================] - 2s 2s/step - loss: -1.7804 - accuracy: 0.5000
Epoch 8/30
1/1 [==============================] - 2s 2s/step - loss: -2.3007 - accuracy: 0.5000
Epoch 9/30
1/1 [==============================] - 2s 2s/step - loss: -2.9518 - accuracy: 0.5000
Epoch 10/30
1/1 [==============================] - 2s 2s/step - loss: -3.6407 - accuracy: 0.5000
E